In [27]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random

In [28]:
HTML("<script src='d3.js'></script>")

In [29]:
df = pd.read_csv("random_data.csv", sep = ',')

In [30]:
df.head(11)

,data,val
0,2010,0
1,2011,71
2,2012,36
3,2013,99
4,2014,85
5,2015,100
6,2016,32
7,2017,93
8,2018,49
9,2019,14


In [31]:
df_dict = df.to_dict(orient="records")
df_dict[:5]

[{'data': 2010, 'val': 0},
 {'data': 2011, 'val': 71},
 {'data': 2012, 'val': 36},
 {'data': 2013, 'val': 99},
 {'data': 2014, 'val': 85}]

In [32]:
js_text_template = Template('''
var margin = {top:20,right:20,bottom:30,left:30};
var width = 720 - margin.left - margin.right;
var height = 375 - margin.top - margin.bottom;

var x = d3.scale.linear().range([0,width]);

var y = d3.scale.linear().range([height,0]);

var color = d3.scale.category10();

var xAxis = d3.svg.axis()
.scale(x)
.orient("bottom");

var yAxis = d3.svg.axis()
.scale(y)
.orient("left");

var svg = d3.select("#$maid").append("svg")
.attr("width", width + margin.left+margin.left)
.attr("height", height + margin.top+margin.bottom)
.append("g")
.attr("transform","translate("+margin.left+","+margin.top+")")

var data = $python_data;

data.forEach(function(d) {
    d.sepalLength = +d.sepalLength;
    d.sepalWidth = +d.sepalWidth;
});


x.domain(d3.extent(data,function(d) { return d.sepalWidth;})).nice();
y.domain(d3.extent(data,function(d) { return d.sepalLength})).nice();

console.log("Extent",d3.extent(data,function(d) { return d.sepalLength}));

svg.append("g")
.attr("class","x axis")
.attr("transform","translate(0,"+height+")")
.call(xAxis)
.append("text")
.attr("class","label")
.attr("x",width)
.style("text-anchor","end")
.text("Sepal Width (cm)");

svg.append("g")
.attr("class","y axis")
.call(yAxis)
.append("text")
.attr("transform","rotate(-90)")
.attr("class","label")
.attr("y",6)
.attr("dy",".71em")
.style("text-anchor","end")
.text("Sepal Length (cm)");

svg.selectAll(".dot")
.data(data)
.enter()
.append("circle")
.attr("class","dot")
.attr("r",3.5)
.attr("cx", function(d) { return x(d.sepalWidth);})
.attr("cy", function(d) { return y(d.sepalLength);})
.style("fill",function(d) { return color(d.species);});

var legend = svg.selectAll(".legend")
.data(color.domain())
.enter().append("g")
.attr("class","legend")
.attr("transform",function(d,i) {return "translate(0,"+i*20+")";});

legend.append("rect")
.attr("x",width-18)
.attr("width",18)
.attr("height",18)
.style("fill",color);

legend.append("text")
.attr("x", width-24)
.attr("y", 9)
.attr("dy",".35em")
.style("text-anchor","end")
.text(function(d) {return d;});

''');

In [40]:
html_template = Template('''
    <style>
    .dot{
        stroke: #000;
    }
    .legend{
        color:blue;
    }
    
    .axis path,
    .axis line{
        fill: none;
        shape-rendering:crispEdges;
    
    }
    </style>
    <div id="scatter-id"></div>
    <script>$js_text</script>
''')

js_text = js_text_template.substitute({
    "python_data":json.dumps(df_dict),
    "maid":"scatter-id"
})

In [41]:
HTML(html_template.substitute({'js_text':js_text}))